# TPC-DS Pipeline  

In [1]:
import tpcds_setup, tpcds_bq, config

In [2]:
from importlib import reload

In [3]:
reload(tpcds_setup)
reload(tpcds_bq)
reload(config)

<module 'config' from '/home/colin/code/bq_snowflake_benchmark/config.py'>

#### I. Make Project Files

In [4]:
tpcds_setup.make_directories()

#### II. Download and Compile TPC-DS Binaries  
1. place unzipped source folder downloaded from TPC in the `download` folder
2. Add the location of the source folder to config.py as the `fp_ds_src` variable  

#### III. Unzip and Rename Source Files

In [6]:
#fp_zip = config.fp_download + config.sep + "tpc-ds_v2.11.0rc2.zip"
tpcds_setup.extract_tpcds_zip(zip_filepath=config.fp_ds_zip,
                              version=config.fp_ds_src_version)

#### IV. Make tpcds

In [7]:
tpcds_setup.make_tpcds()

#### V. Generate the table schema for BigQuery

In [8]:
tpcds_bq.schema(filepath_in=config.tpcds_schema_ansi_sql_filepath, 
                filepath_out=config.tpcds_schema_bq_filepath, 
                dataset_name=config.gcp_bq_table)

1. Generate bash scripts for dsdgen

In [9]:
tpcds_setup.dsdgen_bash_scripts()

In [10]:
tpcds_setup.dsdgen_move_bash_scripts()

### VI. Generate Data  

From the command line, execute appropriate script for CPU size and desired data size.